In [6]:
import datasets
from llama_index.core.schema import Document

guest_dataset = datasets.load_dataset(
    "agents-course/unit3-invitees", 
    split="train"
)

docs = [
    Document(
        text="\n".join([
            f"Name: {guest_dataset['name'][i]}",
            f"Relation: {guest_dataset['relation'][i]}",
            f"Description: {guest_dataset['description'][i]}",
            f"Email: {guest_dataset['email'][i]}",
        ]),
        metadata={
            "name": guest_dataset['name'][i],
            "relation": guest_dataset['relation'][i],
            "description": guest_dataset['description'][i],
            "email": guest_dataset['email'][i]
        }
    )
    for i in range(len(guest_dataset))
]

print(docs[0].text)

Name: Ada Lovelace
Relation: best friend
Description: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine.
Email: ada.lovelace@example.com


In [7]:
# pip install llama-index-retrievers-bm25

In [8]:
from llama_index.core.tools import FunctionTool
from llama_index.retrievers.bm25 import BM25Retriever

bm25_retriever = BM25Retriever.from_defaults(nodes=docs)

def get_guest_info_retriever(query: str) -> str:
    """Retrieves detailed information about gala guests based on their name or relation."""
    results = bm25_retriever.retrieve(query)
    if results:
        return "\n\n".join([doc.text for doc in results[:3]])
    else:
        return "No matching guest information found."

guest_info_tool = FunctionTool.from_defaults(get_guest_info_retriever)

In [9]:
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
import os
from dotenv import load_dotenv

load_dotenv()

hf_token= os.getenv("HF_TOKEN")

llm = HuggingFaceInferenceAPI(
    model_name="Qwen/Qwen2.5-Coder-32B-Instruct",
    temperature=0.7,
    max_tokens=100,
    token=hf_token,
    provider="auto"
)

alfred = AgentWorkflow.from_tools_or_functions(
    [guest_info_tool],
    llm=llm,
)

response = await alfred.run("Tell me about our guest named 'Lady Ada Lovelace'.")

print(" Alfred's Response:")
print(response)

 Alfred's Response:
Lady Ada Lovelace is your best friend. She is an esteemed mathematician and friend, renowned for her pioneering work in mathematics and computing. She is often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine. Her email is ada.lovelace@example.com.
